# Final Aggregation

The results stored in the `experiment_results.json` file shall not be aggregated to a final average only differentiating between the runs with the soft labels and the ones without

## Collect the Data

In [ ]:
import json
import os

# Read the File
json_file_path = "experiment_results.json"

# Initialize data structure
if os.path.exists(json_file_path):
    try:
        with open(json_file_path, 'r') as f:
            data = json.load(f)
    except json.JSONDecodeError:
        data = {}
else:
    data = {}

data

{'mlp': {'gradasc': {'mnist': {'softtargets_False': {'accuracies': [],
     'param_changes': []}}}}}

In [ ]:
# Combine the lists

soft_accs = []
soft_param_changes = []
hard_accs = []
hard_param_changes = []
for arch in data:
    for algo in data[arch]:
        for dset in data[arch][algo]:
            for st_key in data[arch][algo][dset]:
                if "False" in st_key:
                    hard_accs.extend(data[arch][algo][dset][st_key]["accuracies"])
                    hard_param_changes.extend(data[arch][algo][dset][st_key]["param_changes"])
                elif "True" in st_key:
                    soft_accs.extend(data[arch][algo][dset][st_key]["accuracies"])
                    soft_param_changes.extend(data[arch][algo][dset][st_key]["param_changes"])

print(f"{len(soft_accs)} soft target accuracies collected.")
print(f"{len(soft_param_changes)} soft target param changes collected.")
print(f"{len(hard_accs)} hard target accuracies collected.")
print(f"{len(hard_param_changes)} hard target param changes collected.")

Combining entries for mlp, gradasc, mnist, softtargets_False


## Plotting

In [ ]:
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go

# Helper to create or update a distribution plot
def create_dist_plot(data, title, xlabel, color, context_title, existing_fig=None):
    if not data:
        return existing_fig if existing_fig else None
        
    # 1. Calculate Statistics
    mu, std = norm.fit(data)
    
    # 2. Generate PDF points
    x_min, x_max = min(data), max(data)
    # Add some padding to the range
    padding = (x_max - x_min) * 0.2 if x_max != x_min else 0.01
    x_axis = np.linspace(x_min - padding, x_max + padding, 100)
    y_axis = norm.pdf(x_axis, mu, std)
    
    # 3. Create or reuse figure
    if existing_fig:
        fig = existing_fig
        # Optionally update title to indicate multiple distributions
        # fig.update_layout(title=f"{title} (Comparison)")
    else:
        fig = go.Figure()
        fig.update_layout(
            title=f"{title}",
            xaxis_title=xlabel,
            yaxis_title="Probability Density",
            template="plotly_white"
        )
    
    # 4. Add Traces
    # Histogram of actual data
    fig.add_trace(go.Histogram(
        x=data,
        histnorm='probability density',
        name=f'{context_title} (Data)',
        marker_color=color,
        opacity=0.6
    ))
    
    # Normal Distribution Line
    fig.add_trace(go.Scatter(
        x=x_axis,
        y=y_axis,
        mode='lines',
        name=f'{context_title} (Fit: μ={mu:.4f}, σ={std:.4f})',
        line=dict(width=2) # Removed fixed color to allow auto-cycling for comparisons
    ))
    
    return fig

In [ ]:
fig_soft_accs = create_dist_plot(
    soft_accs, 
    "Soft Targets Accuracy Distribution",
    "Accuracy", 
    "blue", 
    "Soft Accuracies"
    )

In [ ]:
fig_soft_param_changes = create_dist_plot(
    soft_param_changes, "Soft Targets Parameter Change Distribution",
    "Parameter Change",
    "green", 
    "Soft Parameter Changes")

In [ ]:
combine_accs = create_dist_plot(
    hard_accs,
    "Hard Targets Accuracy Distribution",
    "Accuracy",
    "red",
    "Hard Accuracies",
    existing_fig=fig_soft_accs)

In [ ]:
combine_param_changes = create_dist_plot(
    hard_param_changes,
    "Hard Targets Parameter Change Distribution",
    "Parameter Change",
    "orange",
    "Hard Parameter Changes",
    existing_fig=fig_soft_param_changes)